In [36]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [ ]:
base_url = "https://www.gs25.com.vn/san-pham/category/thuc-an/"
products = []

for page_num in range(1, 7):
    url = f"{base_url}?true&p={page_num}&group=&sortBy=&sortType="
    print(f"Đang lấy dữ liệu từ: {url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    product_items = soup.select('div.outer-product div.crs div.item')

    if not product_items:
        print(f"Không tìm thấy sản phẩm nào trên trang {page_num}")
        break 

    for item in product_items:
        product_name_tag = item.select_one('h5')
        product_link_tag = item.select_one('a')
        name = product_name_tag.get_text(strip=True) if product_name_tag else 'N/A'
        link = product_link_tag.get('href', 'N/A') if product_link_tag else 'N/A'
        products.append({
            'name': name,
            'link': link,
            'main_description': '',  
            'details_list_html': '', 
            'error': 0              
            })

for product in products:
    print(product)

print(f"\nTổng số sản phẩm tìm thấy: {len(products)}")

Đang lấy dữ liệu từ: https://www.gs25.com.vn/san-pham/category/thuc-an/?true&p=1&group=&sortBy=&sortType=
Đang lấy dữ liệu từ: https://www.gs25.com.vn/san-pham/category/thuc-an/?true&p=2&group=&sortBy=&sortType=
Đang lấy dữ liệu từ: https://www.gs25.com.vn/san-pham/category/thuc-an/?true&p=3&group=&sortBy=&sortType=
Đang lấy dữ liệu từ: https://www.gs25.com.vn/san-pham/category/thuc-an/?true&p=4&group=&sortBy=&sortType=
Đang lấy dữ liệu từ: https://www.gs25.com.vn/san-pham/category/thuc-an/?true&p=5&group=&sortBy=&sortType=
Đang lấy dữ liệu từ: https://www.gs25.com.vn/san-pham/category/thuc-an/?true&p=6&group=&sortBy=&sortType=
{'name': 'Cơm Bò Bulgogi Phô Mai', 'link': 'https://gs25.com.vn/en/san-pham/product/com-bo-bulgogi-pho-mai/', 'main_description': '', 'details_list_html': '', 'error': 0}
{'name': 'Cơm tấm GS25', 'link': 'https://gs25.com.vn/en/san-pham/product/com-tam-gs25/', 'main_description': '', 'details_list_html': '', 'error': 0}
{'name': 'Cơm chiên Hải Sản', 'link': 'htt

In [ ]:
df_products = pd.DataFrame(products)

print(f"\nTổng số sản phẩm: {len(df_products)}")
print("DataFrame danh sách sản phẩm :")
print(df_products.head())
print("-" * 50)


Tổng số sản phẩm tìm thấy ban đầu: 83
DataFrame danh sách sản phẩm (5 dòng đầu):
                     name                                               link  \
0  Cơm Bò Bulgogi Phô Mai  https://gs25.com.vn/en/san-pham/product/com-bo...   
1            Cơm tấm GS25  https://gs25.com.vn/en/san-pham/product/com-ta...   
2       Cơm chiên Hải Sản  https://gs25.com.vn/en/san-pham/product/com-ch...   
3      Cơm chiên bò trứng  https://gs25.com.vn/en/san-pham/product/com-ch...   
4         Cơm Gà Kho gừng  https://gs25.com.vn/en/san-pham/product/com-ga...   

  main_description details_list_html  error  
0                                         0  
1                                         0  
2                                         0  
3                                         0  
4                                         0  
--------------------------------------------------


In [ ]:
for index, row in df_products.iterrows():
    product_url = row['link']
    product_name = row['name']

    if product_url == 'N/A' or not product_url.startswith('http'):
        print(f"Bỏ qua sản phẩm '{product_name}': URL không co ")
        df_products.at[index, 'error'] = 1
        continue

    print(f"Đang lấy chi tiết cho: {product_name} ({product_url})")
    try:
        response = requests.get(product_url, timeout=15)
        response.raise_for_status()
        soup_detail = BeautifulSoup(response.content, 'html.parser')

        div_txt = soup_detail.select_one('div.txt')

        if div_txt:
            main_description_p = None
            paragraphs = div_txt.find_all('p')
            for p in paragraphs:
                p_text = p.get_text(strip=True)
                if p_text and p_text != ' ': 
                    main_description_p = p_text
                    break
            
            df_products.at[index, 'main_description'] = main_description_p if main_description_p else 'N/A'

            ul_tag = div_txt.select_one('ul')
            if ul_tag:
                df_products.at[index, 'details_list_html'] = str(ul_tag)
            else:
                df_products.at[index, 'details_list_html'] = 'N/A'
        else:
            print(f"Không tìm thấy div.txt cho sản phẩm: {product_name} ({product_url})")
            df_products.at[index, 'error'] = 2 
            df_products.at[index, 'main_description'] = 'N/A'
            df_products.at[index, 'details_list_html'] = 'N/A'

    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi lấy chi tiết từ {product_url}: {e}")
        df_products.at[index, 'error'] = 3
        df_products.at[index, 'main_description'] = 'N/A'
        df_products.at[index, 'details_list_html'] = 'N/A'

print("\nDataFrame với mô tả chính và HTML chi tiết ")
print(df_products.head())
print("\nCác cột trong DataFrame cuối cùng:")
print(df_products.columns)
print(f"\nTổng số sản phẩm sau khi lấy chi tiết: {len(df_products)}")


Đang lấy chi tiết cho: Cơm Bò Bulgogi Phô Mai (https://gs25.com.vn/en/san-pham/product/com-bo-bulgogi-pho-mai/)
Đang lấy chi tiết cho: Cơm tấm GS25 (https://gs25.com.vn/en/san-pham/product/com-tam-gs25/)
Đang lấy chi tiết cho: Cơm chiên Hải Sản (https://gs25.com.vn/en/san-pham/product/com-chien-hai-san/)
Đang lấy chi tiết cho: Cơm chiên bò trứng (https://gs25.com.vn/en/san-pham/product/com-chien-bo-trung/)
Đang lấy chi tiết cho: Cơm Gà Kho gừng (https://gs25.com.vn/en/san-pham/product/com-ga-kho-gung/)
Đang lấy chi tiết cho: Cơm gà mắm tỏi (https://gs25.com.vn/en/san-pham/product/com-ga-mam-toi/)
Đang lấy chi tiết cho: Cơm gà sốt Hàn Quốc (https://gs25.com.vn/en/san-pham/product/com-ga-sot-han-quoc/)
Đang lấy chi tiết cho: Cơm thịt bò sốt BBQ (https://gs25.com.vn/en/san-pham/product/com-thit-bo-sot-bbq/)
Đang lấy chi tiết cho: Cơm trộn Hàn Quốc (https://gs25.com.vn/en/san-pham/product/com-tron-han-quoc/)
Đang lấy chi tiết cho: Bánh mì mè Hàn Quốc vị truyền thống (https://gs25.com.vn/en

In [ ]:
print("\nNội dung 'details_list_html' của sản phẩm đầu tiên:")
print(df_products.loc[75, 'details_list_html'])


Nội dung 'details_list_html' của sản phẩm đầu tiên:
<ul data-sourcepos="6:5-10:0">
<li data-sourcepos="6:5-6:48"><strong>Vị:</strong><span> </span>Cay xé lưỡi, béo ngậy của phô mai.</li>
<li data-sourcepos="7:5-7:66"><strong>Đặc trưng:</strong><span> </span>Gà sốt siêu cay, phô mai tan chảy, cơm trắng.</li>
<li data-sourcepos="8:5-8:60"><strong>Phù hợp:</strong><span> </span>Bữa trưa, bữa tối cho người thích ăn cay.</li>
<li data-sourcepos="9:5-10:0"><strong>Ghi chú:</strong><span> </span>Cực kỳ cay, cân nhắc trước khi dùng.</li>
</ul>


In [48]:
df_products['details_list_html']

0     <ul data-end="422" data-start="310">\n<li clas...
1     <ul data-end="427" data-start="315">\n<li clas...
2     <ul data-end="457" data-start="345">\n<li clas...
3     <ul data-end="418" data-start="306">\n<li clas...
4     <ul data-end="423" data-start="311">\n<li clas...
                            ...                        
78                                                  N/A
79                                                  N/A
80                                                  N/A
81                                                  N/A
82                                                  N/A
Name: details_list_html, Length: 83, dtype: object